# Image anamoly detection

https://www.pyimagesearch.com/2020/01/20/intro-to-anomaly-detection-with-opencv-computer-vision-and-scikit-learn/

https://papers.nips.cc/paper/2017/file/9ef2ed4b7fd2c810847ffa5fa85bce38-Paper.pdf

https://paperswithcode.com/method/bigan

https://arxiv.org/pdf/1802.05798.pdf





https://towardsdatascience.com/simplifing-image-outlier-detection-with-alibi-detect-6aea686bf7ba

https://docs.seldon.io/projects/alibi-detect/en/latest/overview/getting_started.html



In [1]:
import h5py
import matplotlib.pyplot as plt
import numpy as np

from helperFunctions import *

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"


In [2]:
fname = "../data/Train/SetA2M.AX.h5"

file = h5py.File(fname, "r")
set_x = file["set_x"][:]
set_y = file["set_y"][:]

train = set_x


fname = "../data/Validation/SetAIA.AX.h5"

file = h5py.File(fname, "r")
set_x = file["set_x"][:]
set_y = file["set_y"][:]

test = set_x



In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob
import os
import shutil
from collections import Counter
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D,\
    Dense, Layer, Reshape, InputLayer, Flatten, Input, MaxPooling2D
from alibi_detect.od import OutlierAE
from alibi_detect.utils.visualize import plot_instance_score, plot_feature_outlier_image


# Memory allocation error fixed using https://www.tensorflow.org/guide/gpu
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print("Error for the fix done for mem allocation error : ",e)



#print all cell contents 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


1 Physical GPUs, 1 Logical GPUs


In [6]:
encoding_dim = 1024
dense_dim = [8, 8, 128]
# dense_dim = [128,128, 3]

encoder_net = tf.keras.Sequential(
  [
      InputLayer(input_shape=train[0].shape),
      Conv2D(64, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(128, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(512, 4, strides=2, padding='same', activation=tf.nn.relu),
      Flatten(),
      Dense(encoding_dim,)
  ])

decoder_net = tf.keras.Sequential(
  [
      InputLayer(input_shape=(encoding_dim,)),
      Dense(np.prod(dense_dim)),      
      Reshape(target_shape=dense_dim),
      Conv2DTranspose(256, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2DTranspose(64, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2DTranspose(3, 4, strides=2, padding='same', activation='sigmoid')
  ])

encoder_net.summary()
decoder_net.summary()
od = OutlierAE( threshold = 0.001,
                encoder_net=encoder_net,
                decoder_net=decoder_net)

adam = tf.keras.optimizers.Adam(lr=1e-4)

od.fit(train, epochs=1, verbose=True,  optimizer = adam)

# od.infer_threshold(test, threshold_perc=95)

# preds = od.predict(test, outlier_type='instance',
#             return_instance_score=True,
#             return_feature_score=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        3136      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       131200    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 512)       1049088   
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              134218752 
Total params: 135,402,176
Trainable params: 135,402,176
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)        

InvalidArgumentError: Incompatible shapes: [64,64,64,3] vs. [64,128,128,3] [Op:SquaredDifference]

In [4]:
train.shape
train_df  = pd.DataFrame(train.reshape((train.shape[0],-1)))
train_df

(1230, 128, 128, 3)

,0,1,2,3,4,5,6,7,8,9,...,49142,49143,49144,49145,49146,49147,49148,49149,49150,49151
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
2,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1226,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1227,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1228,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


In [5]:
test.shape
test_df  = pd.DataFrame(test.reshape((test.shape[0],-1)))
test_df

(4948, 128, 128, 3)

,0,1,2,3,4,5,6,7,8,9,...,49142,49143,49144,49145,49146,49147,49148,49149,49150,49151
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
2,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4943,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4944,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4945,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4946,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses


class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Dense(32, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu")])

    self.decoder = tf.keras.Sequential([
      layers.Dense(16, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(49152, activation="sigmoid")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

In [19]:
autoencoder.compile(optimizer='adam', loss='mae')


In [21]:
history = autoencoder.fit(train_df, train_df, 
          epochs=2, 
          batch_size=512,
           validation_data=(test_df, test_df),
          shuffle=True)


Epoch 1/2
3/3 [==============================] - 1s 233ms/step - loss: 220.8876 - val_loss: 222.9592
Epoch 2/2
3/3 [==============================] - 0s 154ms/step - loss: 220.8731 - val_loss: 222.9516
